In [60]:
from simplegmail import Gmail
from simplegmail.query import construct_query

In [81]:
gmail = Gmail()

In [134]:
emails = gmail.get_unread_inbox()

In [64]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [74]:
model = BertForSequenceClassification.from_pretrained("model")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [140]:
interested_reply = "Thank you for your interest in our product! Here are some details about it..."
not_interested_reply = "Thank you for letting us know. We appreciate your feedback."
prediction_label = {0: "Interested", 1: "Not Interested"}
to_email = 'devastef98@gmail.com'
subject = 'Test Email'

In [158]:
for email in emails:
    sender_email = email.sender
    original_subject = email.subject
    reply_subject = f"Re: {original_subject}"

    content = email.snippet
    inputs = tokenizer(content, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item() # 0 - Interested, 1 - Not Interested
    print(f"Content: {content}")
    print(f"Prediction: {prediction_label[predicted_class]}")
    try:
        if predicted_class == 0:
            reply_body = interested_reply
        else:
            reply_body = not_interested_reply
        gmail.send_message(sender='projectdemo8898@gmail.com', to=sender_email, subject=reply_subject, msg_plain=reply_body)
        gmail.service.users().messages().modify(userId='me', id=email.id, body={'removeLabelIds': ['UNREAD']}).execute() 
        print("Email sent!")
    except Exception as e:
        print(f"Error occurred while sending email: {str(e)}")
    print("-"*50)

Content: May I know the price of this product?
Prediction: Interested
Email sent!
--------------------------------------------------
Content: Hey. I don't want to receive any more e-mails from you. Please stop sending this product details to me.
Prediction: Not Interested
Email sent!
--------------------------------------------------
Content: Hey. I want to know more about this product. Could you please provide more details about this?
Prediction: Interested
Email sent!
--------------------------------------------------
